In [1]:
import xarray as xr
from calculate_exposure_to_haz import *
import numpy as np
exposures_dict = get_exposures(haz_type='TC')


In [2]:
import pandas as pd
import numpy as np
from climada.hazard import Hazard
# Rename columns

unique_years = np.arange(2003,2023)

yearly_hazards = {}

# Loop over each unique year
for year in unique_years:
    file = INPUT_DIR / f"tropical_cyclones/max_monthly/tropical_cyclones_max_monthly_{year}.nc"

    yearly_hazard = xr.open_dataset(file)
    yearly_hazard['longitude'] = yearly_hazard['longitude'].where(yearly_hazard['longitude'] <= 180, yearly_hazard['longitude'] - 360)

    # Convert it into a hazard
    yearly_hazard = Hazard.from_xarray_raster(yearly_hazard, hazard_type='TC', intensity_unit='days', intensity='data')
    # Append the hazard to the list
    yearly_hazards[year] = yearly_hazard



In [3]:
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet

impf_set = ImpactFuncSet()

impfhw = ImpactFunc()
impfhw.id = 1
impfhw.haz_type = 'TC'
impfhw.name = 'Step function tc'
impfhw.intensity_unit = ''
impfhw.intensity = np.array([0, 32, 32.9, 33, 34])
impfhw.mdd = np.array([0, 0, 0, 1, 1])
impfhw.paa = np.ones(impfhw.intensity.shape)
impfhw.check()
impf_set.append(impfhw)

In [4]:
from climada.engine import Impact, ImpactCalc

impact_dict={age:{year:ImpactCalc(exposures_dict[age][str(year)], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}
for age in impact_dict:
    for year in impact_dict[age]:
        impact_dict[age][year].tot_value=0


impact_dict_no_change_vuln_pop={age:{year:ImpactCalc(exposures_dict[age]['2003'], impf_set, yearly_hazards[int(year)]).impact(save_mat=True) for year in exposures_dict[age]} for age in exposures_dict}
for age in impact_dict_no_change_vuln_pop:
    for year in impact_dict_no_change_vuln_pop[age]:
        impact_dict_no_change_vuln_pop[age][year].tot_value=0



2024-08-22 11:44:51,067 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 222116 coordinates.
2024-08-22 11:44:51,173 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 1287 coordinates.
2024-08-22 11:44:53,228 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 222116 coordinates.
2024-08-22 11:44:53,328 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 1287 coordinates.
2024-08-22 11:44:55,382 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 222116 coordinates.
2024-08-22 11:44:55,484 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 1287 coordinates.
2024-08-22 11:44:57,533 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 222116 coordinates.
2024-08-22 11:44:57,638 - climada.util.c

In [5]:
impact_concat = {}
impact_concat_no_change_vuln_pop = {}
for age in impact_dict_no_change_vuln_pop:
    # impact_dict[age].pop(2000)
    # impact_dict[age].pop(2001)
    # impact_dict[age].pop(2002)
    impact_concat[age] = Impact.concat(list(impact_dict[age].values()),reset_event_ids=True)
    impact_concat_no_change_vuln_pop[age] = Impact.concat(list(impact_dict_no_change_vuln_pop[age].values()),reset_event_ids=True)

2024-08-22 11:49:16,325 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:49:16,325 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:49:16,326 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:49:16,327 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-08-22 11:49:16,328 - climada.engine.impact - WARNING - The 

In [6]:
from climada.util import multi_risk
for age in impact_concat_no_change_vuln_pop:

    impact_concat_no_change_vuln_pop[age] = multi_risk.upscale_dates(impact_concat_no_change_vuln_pop[age], by="month")
    impact_concat_no_change_vuln_pop[age] = multi_risk.aggregate_events(impact_concat_no_change_vuln_pop[age], how='max')

In [8]:
for age in impact_concat_no_change_vuln_pop:
    write_impact(impact_concat_no_change_vuln_pop[age], 'TC_ec', age)
    write_impact(impact_concat[age], 'TC', age)